In [1]:
from Bio import Entrez
import matplotlib.pyplot as plt
import matplotlib._color_data as mcd
import pandas as pd
import altair as alt
import math
import random
Entrez.email = "lukas.becker@hhu.de"
overlap = [name for name in mcd.CSS4_COLORS]
overlap.remove("lightgrey")

# Curvibacter sp. AEP1-3 EPS-Operon/Cluster reciprocal BLAST analysis report

The following analysis aims to provide insights into the results of a reciprocal BLAST analysis performed with the celery blast tool (recBLAST). The analysis was conducted with amino acid sequences, which form a putative cluster for the synthesis and export of exopolysaccharides in Curvibacter sp. AEP1-3.

In [2]:
reciprocal_best_hits_file_1 = "../data/curvibacter_rec_blasts/eps_operon_vs_all_complete_chromosome/reciprocal_best_hits_protein_ids.txt"
blastp_fw_table_1 = "../data/curvibacter_rec_blasts/eps_operon_vs_all_complete_chromosome/blastp_fw_out.table"
query_file_1 = "../data/curvibacter_rec_blasts/eps_operon_vs_all_complete_chromosome/curvibacter_aep_eps_operon.faa"

In [3]:
reciprocal_best_hits_file_2 = "../data/curvibacter_rec_blasts/eps_operon_additional_genes_vs_all_complete_chromosome/reciprocal_best_hits_protein_ids.txt"
blastp_fw_table_2 = "../data/curvibacter_rec_blasts/eps_operon_additional_genes_vs_all_complete_chromosome/blastp_fw_out.table"
query_file_2 = "../data/curvibacter_rec_blasts/eps_operon_additional_genes_vs_all_complete_chromosome/additional_eps_loci_sequences.faa"

In [4]:
reciprocal_best_hits_file_3 = "../data/curvibacter_rec_blasts/outer_membrane_proteins_vs_all_complete_chromosome/reciprocal_best_hits_protein_ids.txt"
blastp_fw_table_3 = "../data/curvibacter_rec_blasts/outer_membrane_proteins_vs_all_complete_chromosome/blastp_fw_out.table"
query_file_3 = "../data/curvibacter_rec_blasts/outer_membrane_proteins_vs_all_complete_chromosome/curvibacter_outer_membrane_genes.faa"

In [5]:
reciprocal_best_hits_file_4 = "../data/curvibacter_rec_blasts/glycosyltransferases_vs_all_complete_chromosome/reciprocal_best_hits_protein_ids.txt"
blastp_fw_table_4 = "../data/curvibacter_rec_blasts/glycosyltransferases_vs_all_complete_chromosome/blastp_fw_out.table"
query_file_4 = "../data/curvibacter_rec_blasts/glycosyltransferases_vs_all_complete_chromosome/curvibacter_glycosyltransferases.faa"

In [6]:
rec_data_list = [reciprocal_best_hits_file_1,
                 reciprocal_best_hits_file_2,
                 reciprocal_best_hits_file_3,
                 reciprocal_best_hits_file_4]
fw_blast_data_list = [blastp_fw_table_1,
                      blastp_fw_table_2,
                      blastp_fw_table_3,
                      blastp_fw_table_4]
query_file_data_list = [query_file_1,
                        query_file_2,
                        query_file_3,
                        query_file_4]

In [7]:
#The following code reads in the forward BLAST dataframe and the reciprocal best hits textfile from the recBLAST results
#and returns a pandas dataframe with all orthologous sequences
def extract_orthologous_information(reciprocal_best_hits_file,blastp_fw_table):    
    rec_prot=pd.read_table(reciprocal_best_hits_file)
    fw_res=pd.read_table(blastp_fw_table,header=None)
    fw_res.columns=["qseqid", "sseqid", "evalue", "bitscore", "qgi", "sgi", "sacc", "staxids", "sscinames", "scomnames",
                      "stitle"]

    fw_res['qseqid'] = fw_res['qseqid'].map(lambda line: line.split('.')[0])
    fw_res['sacc'] = fw_res['sacc'].map(lambda line: line.split('.')[0])
    rec_prot = rec_prot.rename(columns={"forward_genome_id": "sacc"})
    rec_prot = rec_prot.rename(columns={"backward_genome_id": "qseqid"})
    result_data = rec_prot.merge(fw_res,how='inner', on=['sacc','qseqid'])
    result_data = result_data.drop_duplicates('sacc', keep='first')
    return result_data

In [8]:
#Adds taxonomic information to pandas dataframe.
#In order to retrieve the taxonomic information biopython calls are conducted with the query accession ids in the pandas dataframe.
#The query file gets processed for extracting the fasta header line, this information is then loaded into a specific dataframe column.
def add_taxonomic_information_to_result_dataframe(result_data,query_file):
    
    def read_query_file(query_file):
        queries = {}
        queryfile = open(query_file, "r")
        for line in queryfile.readlines():
            if ">" in line:
                prot_id = line.split(">")[1].split(' ')[0].split('.')[0]
                line = ' '.join(line.split(">")[1].split(' ')[1:]).rstrip()
                queries[prot_id] = line
        queryfile.close()
        return queries
    
    df = result_data.copy()
    
    queries = read_query_file(query_file)

    dataframes = []
    unique_queries = list(df['qseqid'].unique())
    for query in unique_queries:
        print("processing : {}".format(query))
        dataframe = df.loc[df['qseqid'] == query].copy()
        dataframe['sacc'] = dataframe['sacc'].map(lambda protid: protid.split(".")[0])
        dataframe = dataframe.drop_duplicates(subset=['sacc'], keep="first")
        #print("Current length: {}".format(len(dataframe)))
        if len(dataframe) >= 4999:
            dataframe = dataframe[0:4999]
            print("New Length : {}".format(len(dataframe)))
        staxids=[]
        for ids in list(dataframe['staxids']):
            if type(ids) == str:
                staxids.append(ids.split(";")[0])
            elif type(ids) == int:
                staxids.append(ids)

        result_record = []

        end = len(dataframe[dataframe['qseqid'] == query])
        begin = 0
        step = 500
        steps = 500
        while begin < end:
            if step >= end:
                step = end
            print("\t {} to {}".format(begin,step))
            splitted_ids = staxids[begin:step]
            #range(X) tries for biopython calls
            for attempt in range(10):
                try:
                    print("length ids : {}".format(len(splitted_ids)))
                    handle = Entrez.efetch(id=splitted_ids, db="taxonomy", retmode="xml")
                    record = Entrez.read(handle)
                    handle.close()
                except Exception as e:
                    print("attempt : {} queries : {}".format(attempt,query))
                    if attempt == 9:
                        raise Exception

                else:
                    for rec in record:
                        result_record.append(rec)
                    #print("result record length : {}".format(len(result_record)))
                    break
            begin += steps
            step += steps


        query_info = []
        taxonomy = []
        genus = []
        superfamily = []
        family = []
        #lineageExList = []
        order= []
        classt=[]
        phylum=[]
        for i in range(len(result_record)):
            query_info.append(queries[query])
            taxonomy.append(result_record[i]['ScientificName'])
            #lineageEx = 'linex,'
            for j in result_record[i]['LineageEx']:
                #lineageEx += ','+str(j)
                if j['Rank'] == 'genus':
                    genus.append(j['ScientificName'])
                if j['Rank'] == 'superfamily':
                    superfamily.append(j['ScientificName'])
                if j['Rank'] == 'family':
                    family.append(j['ScientificName'])
                if j['Rank'] == 'order':
                    order.append(j['ScientificName'])
                if j['Rank'] == 'class':
                    classt.append(j['ScientificName'])
                if j['Rank'] == 'phylum':
                    phylum.append(j['ScientificName'])
                    
            #lineageExList.append(lineageEx)
            if (len(taxonomy) != len(genus)):
                genus.append('unknown')
            if (len(taxonomy) != len(superfamily)):
                superfamily.append('unknown')
            if (len(taxonomy) != len(family)):
                family.append('unknown')
            if (len(taxonomy) != len(order)):
                order.append('unknown')
            if (len(taxonomy) != len(phylum)):
                phylum.append('unknown')
            if (len(taxonomy) != len(classt)):
                classt.append('unknown')
        print(len(genus) == len(taxonomy))
        
        # dataframe['taxonomic_name'] = taxonomy
        # compare length of all informations - they should have the same length
        if (len(genus) == len(dataframe) and len(family) == len(dataframe) and len(superfamily) == len(dataframe) and len(
                query_info) == len(dataframe)):
            dataframe['genus'] = genus
            dataframe['superfamily'] = superfamily
            dataframe['family'] = family
            dataframe['order'] = order
            dataframe['phylum'] = phylum
            dataframe['class'] = classt
            dataframe['query_info'] = query_info
            #dataframe['lineageEx'] = lineageExList
        else:
            raise Exception('Informations from biopython calls have not the same length as the dataframe,\
                            dataframe cant get combined with taxonomic information')
        dataframes.append(dataframe)

    result_df = pd.concat(dataframes)

    return result_df

In [9]:
result_df_list = []
for data_index in range(len(rec_data_list)):
    print("[+] iteration step: {}".format(data_index+1))
    result_data = extract_orthologous_information(rec_data_list[data_index],fw_blast_data_list[data_index])
    result_data = add_taxonomic_information_to_result_dataframe(result_data,query_file_data_list[data_index])
    result_df_list.append(result_data)

[+] iteration step: 1
processing : WP_087496569
	 0 to 500
length ids : 500
	 500 to 1000
length ids : 500
	 1000 to 1500
length ids : 500
	 1500 to 1934
length ids : 434
True
processing : WP_087496568
	 0 to 322
length ids : 322
True
processing : WP_087496564
	 0 to 500
length ids : 500
	 500 to 712
length ids : 212
True
processing : WP_087496563
	 0 to 500
length ids : 500
	 500 to 1000
length ids : 500
	 1000 to 1500
length ids : 500
	 1500 to 2000
length ids : 500
	 2000 to 2500
length ids : 500
	 2500 to 3000
length ids : 500
	 3000 to 3500
length ids : 500
	 3500 to 3812
length ids : 312
True
processing : WP_087496562
	 0 to 126
length ids : 126
True
processing : WP_087496561
	 0 to 174
length ids : 174
True
processing : WP_087496560
	 0 to 500
length ids : 500
	 500 to 1000
length ids : 500
	 1000 to 1500
length ids : 500
	 1500 to 2000
length ids : 500
	 2000 to 2500
length ids : 500
	 2500 to 3000
length ids : 500
	 3000 to 3065
length ids : 65
True
processing : WP_087496559
	

	 2500 to 3000
length ids : 500
	 3000 to 3500
length ids : 500
	 3500 to 4000
length ids : 500
	 4000 to 4500
length ids : 500
	 4500 to 4999
length ids : 499
True
processing : WP_087494786
	 0 to 222
length ids : 222
True
processing : WP_157673179
	 0 to 500
length ids : 500
	 500 to 906
length ids : 406
True
processing : WP_087494996
	 0 to 500
length ids : 500
	 500 to 1000
length ids : 500
	 1000 to 1347
length ids : 347
True
processing : WP_087494995
	 0 to 500
length ids : 500
	 500 to 1000
length ids : 500
	 1000 to 1500
length ids : 500
	 1500 to 2000
length ids : 500
	 2000 to 2500
length ids : 500
	 2500 to 3000
length ids : 500
	 3000 to 3244
length ids : 244
True
processing : WP_087494990
	 0 to 500
length ids : 500
	 500 to 1000
length ids : 500
	 1000 to 1425
length ids : 425
True
processing : WP_087495532
	 0 to 500
length ids : 500
	 500 to 1000
length ids : 500
	 1000 to 1500
length ids : 500
	 1500 to 2000
length ids : 500
	 2000 to 2500
length ids : 500
	 2500 to 29

In [10]:
result_df = pd.concat(result_df_list)

In [11]:
#saving pandas dataframe
result_df.to_csv('eps_operon_full.csv',header=list(result_df.columns))

In [14]:
len(result_df)

141770

# Result Dataframe - Columns

The result dataframe has following columns:

- subject accession id
- query sequence id
- subject sequence id
- e-value
- bitscore
- query gi
- subject gi
- subject taxids
- subject scientific names
- subject common names
- subject title
- subject genus
- subject superfamily
- subject family
- subject order
- subject phylum
- subject class
- subject query info

In [16]:
#divide the result dataframe into subframes with query ids
dataframes = []
for subdf in result_df['qseqid'].unique():
    dataframes.append(result_df[result_df['qseqid'] == subdf])

In [17]:
#how many cyanobacteria do possess specific sequences?
for df in dataframes:
    print(list(df['qseqid'].unique())[0],len(df))
    print(len(df[df['phylum'] == 'Cyanobacteria']))

WP_087496569 1934
0
WP_087496568 322
13
WP_087496564 712
29
WP_087496563 3812
17
WP_087496562 126
0
WP_087496561 174
0
WP_087496560 3065
177
WP_087496559 2960
44
WP_087496558 3470
6
WP_087496557 604
44
WP_087496556 153
0
WP_087496555 8517
819
WP_198301848 3022
69
WP_198301976 4078
359
WP_198301975 2878
1
WP_087496554 4615
31
WP_087496553 1647
73
WP_087496552 4249
242
WP_087496550 2064
23
WP_087496547 361
34
WP_087496542 4990
139
WP_198301974 717
0
WP_198301847 64
0
WP_087497039 2447
0
WP_087497040 1740
0
WP_087497092 1739
5
WP_087497118 2311
0
WP_087497441 442
0
WP_157673205 311
0
WP_157673238 6
0
WP_198301832 3096
0
WP_198301912 3233
3
WP_087494701 5488
0
WP_087495532 5810
0
WP_087495344 1239
0
WP_087496545 42
0
WP_157673153 58
7
WP_087495923 655
0
WP_087496544 4053
174
WP_087493790 2190
0
WP_087493791 2434
0
WP_087495343 1971
0
WP_087497333 515
0
WP_198301970 3697
218
WP_087493935 2869
118
WP_087496548 1327
75
WP_087496217 133
5
WP_087495214 4999
908
WP_087494786 222
9
WP_157673179 9

In [34]:
def create_altair_plot_of_taxonomy(clade,dataframes,hide_legend=True):
    result_df = pd.concat(dataframes)
    rf = result_df.drop_duplicates(subset=clade,keep="first")
    bars = []
    
    
    if hide_legend == True:
        for df in dataframes:

            #make_selector = alt.Chart(df).mark_rect().encode(y='genus', color='genus').add_selection(selection)
            bar = alt.Chart(df).mark_bar(tooltip=True).encode(
                alt.Y("count()"),
                alt.X(clade),
                color=alt.Color(clade,legend=None),
                tooltip=['count()','max(bitscore)','min(evalue)',],
            ).interactive().facet(facet='query_info')

            graph = bar
            bars.append(graph)
        
        graphics = bars[0]
        
        
        
    else:
        selection = alt.selection_multi(fields=[clade])
        for df in dataframes:

            #make_selector = alt.Chart(df).mark_rect().encode(y='genus', color='genus').add_selection(selection)
            bar = alt.Chart(df).mark_bar(tooltip=True).encode(
                alt.Y("count()"),
                alt.X(clade),
                color=alt.Color(clade,legend=None),
                tooltip=['count()','max(bitscore)','min(evalue)',],
            ).transform_filter(selection).interactive().facet(facet='query_info')

            graph = bar
            bars.append(graph)
            
        make_selector = alt.Chart(rf).mark_rect().encode(y=clade, color=clade).add_selection(selection)   
        graphics = make_selector | bars[0]
        
    if len(bars) > 1:
        for bar in bars[1:]:
            graphics |= bar
    return graphics

In [36]:
create_altair_plot_of_taxonomy('order',dataframes[:1],hide_legend=False)

alt.HConcatChart(...)

# filtering for species with hits on every input query sequence

1. from above commands we do have access to a list called dataframes, which inherits the result pandas tables for each query sequence

2. lets loop through those dataframes and count the number of unique taxonomic nodes for each query sequence, we can store those numbers in an appropriate dictionary, with taxonomic nodes as keys and the total sum as value

3. from above commands we do have access to a dataframe called result_df, which inherits the full result dataframe (after execution of biopython commands)

In [37]:
#dataframes[0].head()
full_operon_hits = {}
count = 1
for df in dataframes:
    print("[*] Parsing pandas dataframe : {}".format(count))
    for taxonomic_node in list(df['staxids'].unique()):
        if taxonomic_node not in full_operon_hits.keys():
            full_operon_hits[taxonomic_node] = 1
        else:
            full_operon_hits[taxonomic_node] += 1
    print('\t[+] Done parsing df : {}'.format(count))
    count += 1

[*] Parsing pandas dataframe : 1
	[+] Done parsing df : 1
[*] Parsing pandas dataframe : 2
	[+] Done parsing df : 2
[*] Parsing pandas dataframe : 3
	[+] Done parsing df : 3
[*] Parsing pandas dataframe : 4
	[+] Done parsing df : 4
[*] Parsing pandas dataframe : 5
	[+] Done parsing df : 5
[*] Parsing pandas dataframe : 6
	[+] Done parsing df : 6
[*] Parsing pandas dataframe : 7
	[+] Done parsing df : 7
[*] Parsing pandas dataframe : 8
	[+] Done parsing df : 8
[*] Parsing pandas dataframe : 9
	[+] Done parsing df : 9
[*] Parsing pandas dataframe : 10
	[+] Done parsing df : 10
[*] Parsing pandas dataframe : 11
	[+] Done parsing df : 11
[*] Parsing pandas dataframe : 12
	[+] Done parsing df : 12
[*] Parsing pandas dataframe : 13
	[+] Done parsing df : 13
[*] Parsing pandas dataframe : 14
	[+] Done parsing df : 14
[*] Parsing pandas dataframe : 15
	[+] Done parsing df : 15
[*] Parsing pandas dataframe : 16
	[+] Done parsing df : 16
[*] Parsing pandas dataframe : 17
	[+] Done parsing df : 1

In [46]:
# which taxonomic node has the full operon?
target_taxids = []
for hit in full_operon_hits.keys():
    if full_operon_hits[hit] >= 40:
        target_taxids.append(hit)
        print("[+] found : {}".format(hit))

[+] found : 85698
[+] found : 294
[+] found : 2752316
[+] found : 95486
[+] found : 401471
[+] found : 2082385
[+] found : 1844971
[+] found : 296591


In [47]:
full_dataframes = []
for taxid in target_taxids:
    for df in dataframes:
        full_dataframes.append(df[df['staxids'] == taxid])

In [48]:
full_combined_dataframe = pd.concat(full_dataframes)

In [49]:
full_combined_dataframe['sscinames'].unique()

array(['Achromobacter xylosoxidans', 'Pseudomonas fluorescens',
       'Rhodoferax sp. AJA081-3', 'Burkholderia cenocepacia',
       'Undibacterium parvum', 'Polaromonas sp. Pch-P',
       'Curvibacter sp. AEP1-3', 'Polaromonas sp. JS666'], dtype=object)

In [50]:
full_combined_dataframe[full_combined_dataframe['sscinames'] == 'Rhodoferax sp']

,sacc,qseqid,sseqid,evalue,bitscore,qgi,sgi,staxids,sscinames,scomnames,stitle,genus,superfamily,family,order,phylum,class,query_info


# parsing genbank files with Biopython

In [ ]:
from Bio import SeqIO

In [ ]:
from Bio import GenBank

In [ ]:
genebank_file = '../data/curvibacter_eps_operon_17_organisms/rec_blast_curvi_processing/GCF_0004774351_ASM47743v1_genomic.gbff'

In [ ]:
with open(genebank_file) as handle:
    for record in GenBank.parse(handle):
        print(record.accession)

In [ ]:
record.features[100].qualifiers

In [ ]:
for rec in SeqIO.parse(genebank_file,'gb'):
    for feature in rec.features:
        for key, val in feature.qualifiers.items():
            print(key,val)

## How many paralogous sequences reside in the combined dataframe?

1. we can use the target_taxids list to access all relevant taxonomic nodes

In [ ]:
qseqids = list(full_combined_dataframe['qseqid'].unique())
paralogous_dict = {}
for taxid in target_taxids:
    temp_df = full_combined_dataframe[full_combined_dataframe['staxids'] == taxid]
    paralogous_dict[taxid] = []
    for seqid in qseqids:
        paralogous_dict[taxid].append((seqid,len(temp_df[temp_df['qseqid'] == seqid])))

In [ ]:
paralogous_dict

In [ ]:
#lets fetch the taxonomic informations from ncbis taxonomy database
handle = Entrez.efetch(db='taxonomy',id=target_taxids,retmode="xml")
records = Entrez.read(handle)
handle.close()

In [ ]:
for rec in records:
    print("ScientificName : {}\n\t LineageEx __ : ".format(rec['ScientificName']))
    for list_item in rec['LineageEx']:
        print("\t\t{}".format(list_item['ScientificName']))
#records[0]

In [ ]:
dataframes = []
taxids_dict = {}
for taxid in target_taxids:
    taxids_dict[taxid] = None
    dataframes.append(result_df[result_df['staxids'] == taxid])

In [ ]:
for df in dataframes:
    handle = Entrez.efetch(db="protein",id=list(df['sacc']),retmode='xml')
    record = Entrez.read(handle)
    handle.close()
    taxids_dict[str(df['staxids'].unique()[0])] = record

In [ ]:
len(taxids_dict.keys())

In [ ]:
import os
df = pd.concat(dataframes)
for qid in df['qseqid'].unique():
    directory = str(qid)
    os.mkdir(directory)
    target_genes = list(df[df['qseqid'] == qid]['sacc'])
    handle = Entrez.efetch(db='protein',id=target_genes,retmode='xml')
    record = Entrez.read(handle)
    handle.close()
    
    output = open(directory+'/target_fasta_sequences.faa','w')
    for sequence in record:
        GBSeq_locus = sequence['GBSeq_locus']
        GBSeq_definition = sequence['GBSeq_definition']
        organism = sequence['GBSeq_organism']
        taxonomy = sequence['GBSeq_taxonomy']
        seq = sequence['GBSeq_sequence']

        output.write('>'+GBSeq_locus+' '+GBSeq_definition+' '+organism+' '+taxonomy+"\n")
        output.write(seq+"\n")
    output.close()

In [ ]:
for df in dataframes:
    key = str(list(df['staxids'].unique())[0])
    directory = str(key)
    os.mkdir(directory)
    for sequence in taxids_dict[int(key)]:
        output = open(directory+'/'+str(sequence['GBSeq_locus']))
        GBSeq_locus = sequence['GBSeq_locus']
        GBSeq_definition = sequence['GBSeq_definition']
        organism = sequence['GBSeq_organism']
        taxonomy = sequence['GBSeq_taxonomy']
        seq = sequence['GBSeq_sequence']
        
        output.write('>'+GBSeq_locus+' '+GBSeq_definition+' '+organism+' '+taxonomy+"\n")
        output.write(seq+"\n")
        output.close()

In [ ]:
# https://www.ncbi.nlm.nih.gov/bioproject?LinkName=assembly_bioproject&from_uid=9754341 - Rhodoferax sp. AJA081-3
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4053972/ - Candidatus Symbiobacter mobilis CR

In [ ]:
#downloading fasta files based on targets list
def get_sequences_as_fasta(targets,output_filepath,query_name):
    try:
        begin = 0
        step = 500
        steps = 500
        #splits the targets list into chunks of 500 sequences
        while begin < end:
            if step >= end:
                step = end
            print('downloading all hits on {}'.format(query_name))    
            print("\t {} to {}".format(begin,step))
            for attempt in range(10):
                try:
                    print("length ids : {}".format(len(splitted_ids)))
                    handle = Entrez.efetch(db="protein",id=targets,retmode='xml')
                    record = Entrez.read(handle)
                    handle.close()
                except Exception as e:
                    print("attempt : {} queries : {}".format(attempt,query))
                    if attempt == 9:
                        raise Exception('not successfull after 10 attempts')

                else:
                    for rec in record:
                        result_record.append(rec)
                    #print("result record length : {}".format(len(result_record)))
                    break
                    
            begin += steps
            step += steps
        return 0
    except Exception as e:
        raise Exception('Exception occurred : {}'.format(e))

# msa transformation and phylogeny

In [ ]:
tree = "../data/curvibacter_rec_blasts/eps_operon_vs_burkholderiales_cyanobacteria/domain_nj_tree.newick"
msa_file = "../data/curvibacter_rec_blasts/eps_operon_vs_burkholderiales_cyanobacteria/epsA_orthologs.msa"

In [ ]:
ifile = open(msa_file,'r')
count = 1
ofile = open("msa_corrected_header.msa",'w')
for line in ifile.readlines():
    if ">" in line:
        ofile.write(">ID:{}\n".format(count))
        count+=1
    else:
        ofile.write(line)
ofile.close()
ifile.close()

In [ ]:
result_df.head()

In [ ]:
wp_to_taxids = {}
ifile = open(msa_file,'r')
output=open("taxonomy_file.txt",'w')
for line in ifile.readlines():
    if ">" in line:
        wp = line.split(" ")[0].split(">")[1]
        taxonomy = ' '.join(line.split(" ")[1:]).strip()
        taxid = result_df[result_df['sacc'] == wp]['staxids']
        genus = result_df[result_df['sacc'] == wp]['genus'].unique()[0]
        family = result_df[result_df['sacc'] == wp]['family'].unique()[0]
        wp_to_taxids[wp] = (taxid,taxonomy, genus,family)
        output.write(taxonomy+"\n")
output.close()
ifile.close()

In [ ]:
treefile=open(tree,"r")
lines = treefile.readlines()
treefile.close()

In [ ]:
import re

In [ ]:
for wp in wp_to_taxids.keys():
    lines[0] = re.sub(wp,str(wp_to_taxids[wp][0].unique()[0]), lines[0] )

In [ ]:
output=open("annotNewick.newick","w")
output.write(lines[0])
output.close()

# Transcriptome Reciprocal Best Hits

In [ ]:
import pandas as pd
import numpy as np
import itertools as it


def get_seq_match_dict_and_flat_list(df):
    #extract protein identifier for matches: key=protein identifier for forward_input_sequences
    #and values=protein identifier for matches 
    seq_matches_dict = {}
    for i in df[0].unique():
        if "." in i:
            key_id = i.split(".")[0]
            seq_matches_dict[key_id] = list(set(df[df[0] == i][6]))
        else:
            seq_matches_dict[i] = list(set(df[df[0] == i][6]))

    return seq_matches_dict

def extract_reciprocal_best_hits_and_return_protein_ids(seq_matches_backward_dict,seq_matches_forward_dict):
    result_set = []
    for forward_key in seq_matches_forward_dict.keys():
        for forward_value in seq_matches_forward_dict[forward_key]:
            if forward_value in seq_matches_backward_dict.keys():
                if forward_key in seq_matches_backward_dict[forward_value]:
                    result_set.append([forward_value,seq_matches_backward_dict[forward_value]])
    return result_set

In [ ]:
fw_df = '../data/transcriptome_jay/rec_blast/blast_fw_result.table'
bw_df = '../data/transcriptome_jay/rec_blast/blast_bw_result.table'
output = '../data/transcriptome_jay/rec_blast/reciprocal_best_hits.txt'

In [ ]:
fw_results = pd.read_table(fw_df, header=None)
out = open('bw_fasta_queries.faa',"w+")
for gi in fw_results[6][:].unique():
    out.write(gi+"\n")
out.close()

In [ ]:
forward_df = pd.read_table(fw_df,header=None)
forward_df[6] = forward_df[6].map(lambda line : line.split('.')[0])
forward_df = pd.DataFrame([forward_df[0][:],forward_df[6][:]]).transpose()
fw_seqs = get_seq_match_dict_and_flat_list(forward_df)

backward_df = pd.read_table(bw_df,header=None)
backward_df[6] = backward_df[6].map(lambda line : line.split('.')[0])
backward_df = pd.DataFrame([backward_df[0][:],backward_df[6][:]]).transpose()
bw_seqs = get_seq_match_dict_and_flat_list(backward_df)
best_hits = extract_reciprocal_best_hits_and_return_protein_ids(bw_seqs,fw_seqs)

out = open(output,'w')
out.write('forward_genome_id\tbackward_genome_id\n')
for prot_id_pair in best_hits:
    out.write(str(prot_id_pair[0])+'\t'+str(prot_id_pair[1][0])+'\n')
out.close()